# Evaluating a Linear Model
*Curtis Miller*

MSE is a useful metric for seeing the performance of a model, but other metrics can help us decide which model to use.

Here we will use **statsmodels** for fitting linear models since the package easily computes the metrics I want to see.

Let's load in the Boston housing dataset.

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [2]:
boston_obj = load_boston()
data_train, data_test, price_train, price_test = train_test_split(boston_obj.data, boston_obj.target)

## Fitting a Linear Model with OLS in **statsmodels**

The `OLS` object in **statsmodels** handles fitting models with OLS. Below I show how to fit the same model for Boston home prices I fitted using **scikit-learn**.

In [3]:
import statsmodels.api as sm
import numpy as np

In [4]:
data_train, data_test = sm.add_constant(data_train), sm.add_constant(data_test)    # Necessary to add the intercept
data_train[:5, :]

array([[1.00000e+00, 7.67202e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00,
        6.93000e-01, 5.74700e+00, 9.89000e+01, 1.63340e+00, 2.40000e+01,
        6.66000e+02, 2.02000e+01, 3.93100e+02, 1.99200e+01],
       [1.00000e+00, 6.91100e-02, 4.50000e+01, 3.44000e+00, 0.00000e+00,
        4.37000e-01, 6.73900e+00, 3.08000e+01, 6.47980e+00, 5.00000e+00,
        3.98000e+02, 1.52000e+01, 3.89710e+02, 4.69000e+00],
       [1.00000e+00, 3.68940e-01, 2.20000e+01, 5.86000e+00, 0.00000e+00,
        4.31000e-01, 8.25900e+00, 8.40000e+00, 8.90670e+00, 7.00000e+00,
        3.30000e+02, 1.91000e+01, 3.96900e+02, 3.54000e+00],
       [1.00000e+00, 7.61620e-01, 2.00000e+01, 3.97000e+00, 0.00000e+00,
        6.47000e-01, 5.56000e+00, 6.28000e+01, 1.98650e+00, 5.00000e+00,
        2.64000e+02, 1.30000e+01, 3.92400e+02, 1.04500e+01],
       [1.00000e+00, 1.51902e+00, 0.00000e+00, 1.95800e+01, 1.00000e+00,
        6.05000e-01, 8.37500e+00, 9.39000e+01, 2.16200e+00, 5.00000e+00,
        4.03000e+02, 1.470

In [5]:
data_train[:5, 0]

array([1., 1., 1., 1., 1.])

In [6]:
ols1 = sm.OLS(price_train, data_train)    # Target, features
model1 = ols1.fit()
model1.params    # The parameters of the regression

array([ 3.72034520e+01, -1.08373524e-01,  5.32398554e-02,  7.13335371e-02,
        3.19363160e+00, -2.13738034e+01,  3.76094998e+00,  3.56314642e-03,
       -1.42530066e+00,  2.84001139e-01, -1.12664749e-02, -9.15548827e-01,
        7.55580272e-03, -5.13539339e-01])

In [7]:
model1.predict([[    # An example prediction
    1,      # Intercept term; always 1
    10,     # Per capita crime rate
    25,     # Proportion of land zoned for large homes
    5,      # Proportion of land zoned for non-retail business
    1,      # Tract bounds the Charles River
    0.3,    # NOX concentration
    10,     # Average number of rooms per dwelling
    2,      # Proportion of owner-occupied units built prior to 1940
    10,     # Weighted distance to employment centers
    3,      # Index for highway accessibility
    400,    # Tax rate
    15,     # Pupil/teacher ratio
    200,    # Index for number of blacks
    5       # % lower status of population
]])

array([39.50813578])

We can get a summary of the model easily in **statsmodels**.

In [8]:
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.725
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     74.03
Date:                Mon, 06 May 2019   Prob (F-statistic):           8.44e-94
Time:                        09:12:09   Log-Likelihood:                -1132.3
No. Observations:                 379   AIC:                             2293.
Df Residuals:                     365   BIC:                             2348.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         37.2035      6.003      6.197      0.0

Let's parse these results:

* `R-squared` (mathematically, $R^2$) describes how much variation in the target variable the model is able to "explain." (Think: predict.) Practitioners prefer `Adj. R-squared` ($\bar{R}^2$) since this takes into account how many variables are used. (it is impossible for $R^2$ to go down when adding variables even if those variables only contribute noise; $\bar{R}^2$ doesn't have this property.) Here $\bar{R}^2$ is somewhat high, suggesting that the model does a reasonable job at predicting home prices.
* `F-statistic` is the test statistic for a statistical test to determine if any coefficients in the model are not zero. `Prob (F-statistic)` is the corresponding $p$-value. Here the model clearly has a non-zero coefficient, though the statistic does not say which.
* `AIC` is the **Akaike information criterion (AIC)**, and `BIC` the **Bayesian information criterion (BIC)**. These are measures of the quality of statistical models and provide a means to decide between models. Models that lead to smaller AIC and BIC are seen as better.
* The table contains the coefficients of the statistical model and the results of $t$-tests to determine if the coefficients are zero or not, in addition to confidence intervals for the coefficient values. We might consider removing features with coefficients not statistically different from zero from our model (but we should also refer to the AIC and BIC statistics when making decisions between models).

## Using AIC to Pick Models

Let's see how we can use the AIC to decide between different models. (The BIC can be used similarly.) Notice that in our table the third and seventh features don't have coefficients statistically different from zero (these correspond to proportion of non-retail business acres per town and proportion of owner-occupied units built prior to 1940). Does a model without these features do better according to the AIC?

In [9]:
ols2 = sm.OLS(price_train, np.delete(data_train, [3, 7], axis=1))
model2 = ols2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     87.63
Date:                Mon, 06 May 2019   Prob (F-statistic):           1.59e-95
Time:                        09:12:09   Log-Likelihood:                -1132.8
No. Observations:                 379   AIC:                             2290.
Df Residuals:                     367   BIC:                             2337.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.6868      5.958      6.157      0.0

This model has a smaller AIC.

The different AIC values can be interpreted this way: If model 1 has $\text{AIC}_1$ and model 2 $\text{AIC}_2$ and $\text{AIC}_2 < \text{AIC}_1$, then model 1 is $\exp((\text{AIC}_2 - \text{AIC}_1)/2)$ times more likely to minimize information loss than model 2.

In [10]:
np.exp((model2.aic - model1.aic)/2)

0.23044709840835434

The inverse of that quantity can be interpreted as how much more likely model 2 is to minimize the information loss than model 1.

In [11]:
np.exp((model1.aic - model2.aic)/2)

4.3393907187670075

So we can see that model 2 should be preferred to model 1.

Let's see how model 2 would have done on the test set, evaluating its performance with the MSE.

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
price_train_pred = model2.predict(np.delete(data_train, [3, 7], axis=1))
mean_squared_error(price_train, price_train_pred)     # Performance on the training set

23.10725806084707

In [18]:
price_test_pred = model2.predict(np.delete(data_test, [3, 7], axis=1))
mean_squared_error(price_test, price_test_pred)     # Performance on the training set

18.820105467553404

In comparison to model 1...

In [15]:
price_test_pred_mod1 = model1.predict(data_test)
mean_squared_error(price_test, price_test_pred_mod1)

19.147546993735563

Model 2 did better.